# Census for Academic Jobs

## Setup

In [7]:
# need to install wikia api
!pip install wikia tqdm pandas bs4 -q

In [8]:
import os,sys
import pandas as pd
import wikia
from wikia import WikiaError
from tqdm import tqdm
from datetime import datetime
import bs4

In [9]:
# some constants
WIKI_NAME = 'academicjobs'
MAIN_PAGE_NAME = 'Academic_Jobs_Wiki'

## Find relevant pages

### Get the top level discipline pages

In [10]:
# what year is it
now=datetime.now()
the_year = now.year

In [11]:
# earliest year on the main page is 2007, but the formatting is standardized around 2011
years = list(range(2011, the_year + 1))
years

[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [12]:
# Change for history?
disc_name = 'English Literature'

In [28]:
# get pages for discipline
disc_pages=[f'{disc_name} {year}-{year+1}' for year in years]

# fix for 2013
disc_pages = [x.replace('English Literature 2013-2014','English Literature 2013-14') for x in disc_pages]

# fix for 2020
disc_pages += ['Ethnic Studies 2020-2021']
years += [2020]

# disc_pages,years
# for dp in disc_pages:
#     print(f'[{dp.split()[-1].split("-")[0]}](https://academicjobs.wikia.org/wiki/{dp.replace(" ","_")})', end=' | ')

### Get links from discipline pages

In [14]:
def get_links_from_disc_page(disc_page_name,year):
    # get page from wikia
    page=wikia.page(WIKI_NAME, disc_page_name)
    
    # get html
    html = page.html()
    
    # read html
    dom = bs4.BeautifulSoup(html)
    
    # get links
    links=[]
    
    for link in dom('a'):
        try:
            href=link['href']
        except KeyError:
            continue
        if not '/wiki/' in href: continue
        wikilink=href.split('/wiki/')[1]
        if ':' in wikilink or '?' in wikilink: continue
        if wikilink==disc_page_name.replace(' ','_'): continue
        if year and (not str(year) in wikilink and not str(year+1) in wikilink) or (str(int(year)-1) in wikilink): continue
        
        if not wikilink in links:
            links+=[wikilink]
    
    return links

In [15]:
# get_links_from_disc_page('Ethnic_Studies_2020-2021',year=2020)

In [16]:
# Get all links

def strip_year_from_page(page_name):
    pdat=page_name.split('_')
    return ' '.join(pdat[:-1])

def get_all_links():
    link_ld=[]
    for dpage,dyear in tqdm(list(zip(disc_pages,years))):
        links=get_links_from_disc_page(dpage, year=dyear)
        for link in links:
            link_d={'disc_page':dpage, 'year':dyear, 'page':link, 'page_group':strip_year_from_page(link)}
            link_ld.append(link_d)

    return link_ld

In [17]:
LINK_LD = get_all_links()
len(LINK_LD)

100%|██████████| 11/11 [00:08<00:00,  1.32it/s]


334

### Clean links

In [18]:
df_pages=pd.DataFrame(LINK_LD)
df_pages

,disc_page,year,page,page_group
0,English Literature 2011-2012,2011,English_Literature_2012-2013,English Literature
1,English Literature 2011-2012,2011,English_Lit_Salaries_2011-2012,English Lit Salaries
2,English Literature 2011-2012,2011,Medieval_2011,Medieval
3,English Literature 2011-2012,2011,Renaissance_2011,Renaissance
4,English Literature 2011-2012,2011,Restoration/18th-Century_2011,Restoration/18th-Century
...,...,...,...,...
329,Ethnic Studies 2020-2021,2020,"South_Asian,_Indian_Ocean,_%26_Himalayan_Studi...","South Asian, Indian Ocean, %26 Himalayan Studi..."
330,Ethnic Studies 2020-2021,2020,African_%26_African_American_Studies_2020-2021,African %26 African American Studies
331,Ethnic Studies 2020-2021,2020,Asian_%26_Asian_American_Studies_2020-2021,Asian %26 Asian American Studies
332,Ethnic Studies 2020-2021,2020,Jewish_Studies_2020-2021,Jewish Studies


In [286]:
# Bug where some pages are double counted!?
df_pages = df_pages.drop_duplicates('page',keep='last')
df_pages

,disc_page,year,page,page_group
0,English Literature 2011-2012,2011,English_Literature_2012-2013,English Literature
1,English Literature 2011-2012,2011,English_Lit_Salaries_2011-2012,English Lit Salaries
2,English Literature 2011-2012,2011,Medieval_2011,Medieval
3,English Literature 2011-2012,2011,Renaissance_2011,Renaissance
4,English Literature 2011-2012,2011,Restoration/18th-Century_2011,Restoration/18th-Century
...,...,...,...,...
330,Ethnic Studies 2020-2021,2020,"South_Asian,_Indian_Ocean,_%26_Himalayan_Studi...","South Asian, Indian Ocean, %26 Himalayan Studi..."
331,Ethnic Studies 2020-2021,2020,African_%26_African_American_Studies_2020-2021,African %26 African American Studies
332,Ethnic Studies 2020-2021,2020,Asian_%26_Asian_American_Studies_2020-2021,Asian %26 Asian American Studies
333,Ethnic Studies 2020-2021,2020,Jewish_Studies_2020-2021,Jewish Studies


In [287]:
# Testing
# df_pages[df_pages.page.str.contains('African')]

### Cleaning page names

## Step 3: Processing pages

In [288]:
def decide_if_school(title):
    title=str(title)
    #if title in not_unis: return 'n'
    if 'College' in title: return 'y'
    if 'Universit' in title: return 'y'
    if 'UC ' in title: return 'y'
    if 'Demographics' in title: return 'n'
    if 'State' in title: return 'y'
    if any([ (w.startswith('(') and w.endswith(')') and w.upper()==w)   for w in title.split()]): return 'y'
    if '<b>' in title: return 'n'
    return ''

In [289]:
def decide_if_tt(title,ad,nowtt):
    if ' TT ' in title: return 'y'
    if 'Lecturer' in title.split() or 'Visiting Assistant Professor' in title: return 'n'
    if nowtt is not None: return 'y' if nowtt else 'n'

    if "Visiting Assistant Professor" in ad: return 'n'
    if "tenure-track" in ad.lower() or "tenure Track" in ad.lower(): return "y"
    if "Assistant Professor" in ad or "Associate Professor" in ad or "Full Professor" in ad: return "y"
    
    return ''

In [290]:
def decide_job_type(IsTT,page_name):
    if IsTT=='y': return 'TT'
    if 'Postdoc' in page_name: return 'Postdoc'
    if IsTT=='n': return 'Non-TT'
    return 'Unknown'

In [291]:
bad_domains = ['bit.ly','fandom','youtube']

def parse_section(section_dom,section_title,now_isTT,page_name):
    section_content=section_dom.text.replace('Edit\n','')
    #print('\n'*5)
    
    from urllib.parse import urlsplit
    
    links = []
    for a in section_dom('a'):
        try:
            href=a['href']
        except KeyError:
            continue
            
        if '/wiki/' in href: continue
        urldat=urlsplit(href)
        if not urldat.path: continue
        #link=urldat.netloc + urldat.path
        link=href
        
        if any([domain in link for domain in bad_domains]): continue
        links+=[link]
    
    if not section_content: return
    
    # save data for this job
    row = {}
    row['section_content'] = section_content.replace('[edit | edit source]','').strip().replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n')
    row['section_links'] = ' | '.join(links)
    row['section_title'] = bs4.BeautifulSoup(section_title).text
    row['IsTT'] = decide_if_tt(row['section_title'], row['section_content'], now_isTT)
    row['IsUni'] = decide_if_school(section_title) # if row['IsTT']!='y' else 'y'
    row['JobType'] = decide_job_type(row['IsTT'], page_name)
    row['JobID'] = row['section_links'] if row['section_links'] else row['section_title']
    return row


def process_page(page_name):
    # get page
    page_name_q = page_name.replace('%26','&').replace('%27',"'")
    page_name_safe = page_name_q.replace('/','_')
    cachefn=f'cache/{page_name_safe}.html'
    if not os.path.exists(cachefn):
        page = wikia.page(WIKI_NAME, page_name_q)
        # get html
        html=page.html()
    else:
#         print('Using cache')
        with open(cachefn) as f: html=f.read()
    
    
    # start data
    data = []
    now_isTT=None
    for section in html.split('mw-headline')[1:]:
        section_title=section.split('</span>')[0].split('">')[-1].strip()
        if 'RECENT ACTIVITY' in section_title: continue
        if 'Demographics' in section_title: continue
        if 'Instructions' in section_title: continue
        if 'Word on the Street' in section_title: continue
        if 'Tenure-Track Positions' in section_title:
            now_isTT=True
            continue
        if 'Visiting Positions' in section_title:
            now_isTT=False
            continue
        if section_title.startswith('Humanities and Social Sciences Postdocs'):
            #now_isTT=False
            continue
        
        if page_name in {'Spanish_and_Portuguese_2020-2021','Film_Studies_2020-2021','French_%26_Francophone_2020-2021'}:
            # these use a different format for no reason!
            for ol in bs4.BeautifulSoup(section)('ol'):
                for p in ol('li'):
                    if '<b>' in str(p):
                        section_title_p=list(p('b'))[0].text
                        row=parse_section(p,section_title_p,None,page_name)
                        data.append(row)
        elif section_title.startswith('Jobs with 2020') or section_title.startswith('Jobs with 2021'):   # 2020 changed format!?
            for p in bs4.BeautifulSoup(section)('p'):
                if '<b>' in str(p):
                    section_title_p=list(p('b'))[0].text
                    row=parse_section(p,section_title_p,None,page_name)
                    data.append(row)
        else:
            section_dom=bs4.BeautifulSoup(section.split('</span>',1)[-1])
            row=parse_section(section_dom,section_title,now_isTT,page_name)
            data.append(row)
    return [d for d in data if d]

In [292]:
#process_page('Restoration_/_18th_Century_British_2020-2021')

In [293]:
# process_page('French_%26_Francophone_2020-2021')

In [294]:
# process_page('Restoration_/_18th_Century_British_2019-2020')

In [295]:
# process_page('Comparative_Literature_2020-2021')
# process_page('Environmental_Literature_2020-2021')

In [296]:
# # process_page('African_%26_African_American_Studies_2020-2021')
# res=process_page('Early_Modern_/_Renaissance_2020-2021')
# for d in res: print(d['section_title'])

In [297]:
# res=process_page('Spanish_and_Portuguese_2020-2021')
# for d in res: print(d['section_title'])

## Sanity check for 2020 (formatting changed)

Bad pages:
* Early_Modern_/_Renaissance_2020-2021 --> fixed by caching


In [298]:
pages_2020=df_pages[df_pages.year==2020].page
# len(pages_2020)

In [299]:
# ok, looks good for now (11-23-2020)
# uncomment to check:

# for page in pages_2020:
#     try:
#         res=process_page(page)
#         print(page,[d['section_title'] for d in res],'\n')
#     except (IndexError,WikiaError) as e:
#         print(page,'!!!',e)

## Step 4: Gathering all pages' data

In [300]:
# Get all pages' data!!!!
def get_all_data():
    data_ld=[]
    all_pages=list(df_pages.page)
    for i,page in enumerate(tqdm(sorted(all_pages))):
        try:
            page_data = process_page(page)
        except WikiaError as e:
            continue
        if not page_data: continue
        #datadx={**page_data, **{'page':page}}
        for dx in page_data:
            if not dx: continue
            dx['page']=page
            data_ld.append(dx)
    return pd.DataFrame(data_ld)

In [301]:
# Big data crunching step!
df_data=get_all_data()
df_data

100%|██████████| 316/316 [01:18<00:00,  4.03it/s]


,section_content,section_links,section_title,IsTT,IsUni,JobType,JobID,page
0,,,20-21 c. American Jobs 2011,,n,Unknown,20-21 c. American Jobs 2011,20-21_c._American_2011
1,"Link\n""Tenure-track position in American liter...",http://agnesscott.edu/about/employment/faculty...,Agnes Scott (GA),y,y,TT,http://agnesscott.edu/about/employment/faculty...,20-21_c._American_2011
2,MLA JIL:\nFull-time position available beginni...,mailto:cynthia.appl@bakeru.edu,Baker University (KS),,y,Unknown,mailto:cynthia.appl@bakeru.edu,20-21_c._American_2011
3,From MLA JIL (2/25)\nThe Department of English...,mailto:bcenglish@beloit.edu,Beloit College (VAP position),,y,Unknown,mailto:bcenglish@beloit.edu,20-21_c._American_2011
4,Bethany College invites applications for a One...,http://www.higheredjobs.com/faculty/details.cf...,Bethany College (VAP - American Lit / Poetry W...,,y,Unknown,http://www.higheredjobs.com/faculty/details.cf...,20-21_c._American_2011
...,...,...,...,...,...,...,...,...
15648,Women’s & Gender Studies at the University of ...,https://jobs.uwm.edu/postings/28229,University of Wisconsin-Milwaukee - Visiting A...,n,y,Non-TT,https://jobs.uwm.edu/postings/28229,Womens/Gender/Queer_Studies_2018-2019
15649,The Program in American Culture Studies (AMCS)...,https://sites.wustl.edu/amcspostdoc/information/,Washington University in St. Louis (MO) - Post...,n,y,Non-TT,https://sites.wustl.edu/amcspostdoc/information/,Womens/Gender/Queer_Studies_2018-2019
15650,The Program in American Culture Studies (AMCS)...,https://sites.wustl.edu/amcspostdoc/information/,Washington University in St. Louis (MO) - Post...,n,y,Non-TT,https://sites.wustl.edu/amcspostdoc/information/,Womens/Gender/Queer_Studies_2018-2019
15651,,,Administrative Positions,n,,Non-TT,Administrative Positions,Womens/Gender/Queer_Studies_2018-2019


## Step 5: Postprocessing

In [309]:
df = df_pages.merge(df_data,on='page') #.merge(df_aliases,on='page_group')

### Sanity checks

In [310]:
df[(df.page_group=='Comparative Literature') & (df.year==2020)]

,disc_page,year,page,page_group,section_content,section_links,section_title,IsTT,IsUni,JobType,JobID
15529,Ethnic Studies 2020-2021,2020,Comparative_Literature_2020-2021,Comparative Literature,"UC Berkeley (Berkeley, CA). Assistant Professo...",https://aprecruit.berkeley.edu/JPF02679,"UC Berkeley (Berkeley, CA).",y,y,TT,https://aprecruit.berkeley.edu/JPF02679
15530,Ethnic Studies 2020-2021,2020,Comparative_Literature_2020-2021,Comparative Literature,Université de Caen (FRA:CAL). Open Rank Profes...,https://euraxess.ec.europa.eu/jobs/537836,Université de Caen (FRA:CAL).,,y,Unknown,https://euraxess.ec.europa.eu/jobs/537836
15531,Ethnic Studies 2020-2021,2020,Comparative_Literature_2020-2021,Comparative Literature,"Cambridge University (Cambridge, UK): Universi...",https://www.jobs.cam.ac.uk/job/27641/,"Cambridge University (Cambridge, UK)",,y,Unknown,https://www.jobs.cam.ac.uk/job/27641/
15532,Ethnic Studies 2020-2021,2020,Comparative_Literature_2020-2021,Comparative Literature,"Stanford University (Stanford, CA). Postdoctor...",https://gender.stanford.edu/fellowships/postdo...,"Stanford University (Stanford, CA).",,y,Unknown,https://gender.stanford.edu/fellowships/postdo...


In [312]:
df.IsUni.value_counts()

y    14241
      1055
n      357
Name: IsUni, dtype: int64

In [315]:
#df[df.IsUni==''].section_title

In [316]:
df.IsTT.value_counts()

y    10134
      3923
n     1596
Name: IsTT, dtype: int64

In [317]:
# df.alias.value_counts().iloc[:5]

In [320]:
df.JobType.value_counts()

TT         10134
Postdoc     2456
Non-TT      1577
Unknown     1486
Name: JobType, dtype: int64

In [321]:
date=f'{now.year}-{now.month}-{now.day}'
ofn=f'data.jobcensus.wiki.{date}.csv'
df.to_csv(ofn,index=False)